In [1]:
import tensorflow as tf 
import numpy as np 
from tensorflow import keras 
import os 
import math 
import random 
import pickle 
import matplotlib.pyplot as plt 
from collections import deque 

from vehicle_model_DDQN3_1 import Environment 
from cell_model import CellModel 

os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

In [2]:
drving_cycle = '../../OC_SIM_DB/OC_SIM_DB_Cycles/Highway/01_FTP72_fuds.mat'
battery_path = "../../OC_SIM_DB/OC_SIM_DB_Bat/OC_SIM_DB_Bat_nimh_6_240_panasonic_MY01_Prius.mat"
motor_path = "../../OC_SIM_DB/OC_SIM_DB_Mot/OC_SIM_DB_Mot_pm_95_145_X2.mat"
cell_model = CellModel()
env = Environment(cell_model, drving_cycle, battery_path, motor_path, 1)


In [3]:
# STATE_SIZE = env.calculation_comp["state_size"]
STATE_SIZE = 4
ACTION_SIZE = env.calculation_comp["action_size"] 
LEARNING_RATE = 0.00025 

TOTAL_EPISODES = 200
MAX_STEPS = 50000 

GAMMA = 0.95 

MAX_EPSILON = 1 
MIN_EPSILON = 0.01 
DECAY_RATE = 0.00002
BATCH_SIZE = 32 
TAU = 0.001 
DELAY_TRAINING = 3000 
EPSILON_MIN_ITER = 5000

In [4]:
primary_network = keras.Sequential([
    keras.layers.Dense(30, activation="relu", kernel_initializer=keras.initializers.he_normal()), 
#     keras.layers.BatchNormalization(), 
    keras.layers.Dense(30, activation="relu", kernel_initializer=keras.initializers.he_normal()),
#     keras.layers.BatchNormalization(), 
    keras.layers.Dense(ACTION_SIZE),
])
target_network = keras.Sequential([
    keras.layers.Dense(30, activation="relu", kernel_initializer=keras.initializers.he_normal()), 
#     keras.layers.BatchNormalization(), 
    keras.layers.Dense(30, activation="relu", kernel_initializer=keras.initializers.he_normal()),
#     keras.layers.BatchNormalization(), 
    keras.layers.Dense(ACTION_SIZE),
])

primary_network.compile(
    loss="mse", 
    optimizer=keras.optimizers.Adam(lr=LEARNING_RATE) 
)

# for t, p in zip(target_network.trainable_variables, primary_network.trainable_variables): 
#     t.assign(p)

In [5]:
def update_network(primary_network, target_network): 
    for t, p in zip(target_network.trainable_variables, primary_network.trainable_variables): 
        t.assign(t * (1 - TAU) + p * TAU)

In [6]:
class Memory: 
    def __init__(self, max_memory): 
        self.max_memory = max_memory 
        self._samples = [] 
        
    def add_sample(self, sample): 
        self._samples.append(sample)
        if len(self._samples) > self.max_memory: 
            self._samples.pop(0)
        
    def sample(self, no_samples): 
        if no_samples > len(self._samples): 
            return random.sample(self._samples, len(self._samples))
        else: 
            return random.sample(self._samples, no_samples)
    
    @property
    def num_samples(self):
        return len(self._samples)
    

# memory = Memory(10000)

In [7]:
def choose_action(state, primary_network, eps): 
    if random.random() < eps: 
        return random.randint(0, ACTION_SIZE - 1)
    else: 
        return np.argmax(primary_network(np.array(state).reshape(1, -1))) 

In [8]:
def train(primary_network, target_network, memory): 
    batch = memory.sample(BATCH_SIZE)
    states = np.array([val[0] for val in batch]) 
    actions = np.array([val[1] for val in batch])
    rewards = np.array([val[2] for val in batch])
    next_states = np.array([np.zeros(STATE_SIZE) if val[3] is None else val[3]  
                            for val in batch])
    
    prim_qt = primary_network(states)
    prim_qtp1 = primary_network(next_states)
    target_q = prim_qt.numpy() 
    updates = rewards 
    valid_idxs = next_states.sum(axis=1) != 0 
    batch_idxs = np.arange(BATCH_SIZE)
    prim_action_tp1 = np.argmax(prim_qtp1.numpy(), axis=1)
    q_from_target = target_network(next_states)
    updates[valid_idxs] += GAMMA * q_from_target.numpy()[batch_idxs[valid_idxs], 
                                                        prim_action_tp1[valid_idxs]]
    
    target_q[batch_idxs, actions] = updates 
    loss = primary_network.train_on_batch(states, target_q)
    return loss 
    
    
    

In [9]:
def initialization_with_rewardFactor(reward_factor):
    env = Environment(cell_model, drving_cycle, battery_path, motor_path, reward_factor)
    
    memory = Memory(10000)
    
    primary_network = keras.Sequential([
        keras.layers.Dense(30, activation="relu", kernel_initializer=keras.initializers.he_normal()),
#         keras.layers.BatchNormalization(),  
        keras.layers.Dense(30, activation="relu", kernel_initializer=keras.initializers.he_normal()),
#         keras.layers.BatchNormalization(), 
        keras.layers.Dense(ACTION_SIZE),
    ])
    target_network = keras.Sequential([
        keras.layers.Dense(30, activation="relu", kernel_initializer=keras.initializers.he_normal()), 
#         keras.layers.BatchNormalization(), 
        keras.layers.Dense(30, activation="relu", kernel_initializer=keras.initializers.he_normal()),
#         keras.layers.BatchNormalization(), 
        keras.layers.Dense(ACTION_SIZE),
    ])
    primary_network.compile(
        loss="mse", 
        optimizer=keras.optimizers.Adam(lr=LEARNING_RATE) 
    )
    return env, memory, primary_network, target_network 
    

In [10]:
print("environment version: {}".format(env.version)) 

 
reward_factors = [10]
results_dict = {} 

for reward_factor in reward_factors: 
    eps = MAX_EPSILON 
    steps = 0
    episode_rewards = [] 
    episode_SOCs = [] 
    episode_FCs = [] 
    
    env, memory, primary_network, target_network = initialization_with_rewardFactor(reward_factor)
    for episode in range(TOTAL_EPISODES): 
        state = env.reset() 
        avg_loss = 0 
        total_reward = 0
        cnt = 1 

        while True:
            action = choose_action(state, primary_network, eps)
            next_state, reward, done = env.step(action)
            total_reward += reward 
            if done: 
                next_state = None 
            memory.add_sample((state, action, reward, next_state))

            if steps > DELAY_TRAINING: 
                loss = train(primary_network, target_network, memory)
                update_network(primary_network, target_network)
                eps = MIN_EPSILON + (MAX_EPSILON - MIN_EPSILON) * np.exp(-DECAY_RATE * steps)
            else: 
                loss = -1

            avg_loss += loss 
            steps += 1 

            if done: 
                if steps > DELAY_TRAINING: 
                    SOC_deviation_history = np.sum(np.abs(np.array(env.history["SOC"]) - 0.6)) 
                    avg_loss /= cnt 
                    print('Episode: {}'.format(episode + 1),
                          'Total reward: {}'.format(total_reward), 
                          'Explore P: {:.4f}'.format(eps), 
                          "SOC: {:.4f}".format(env.SOC), 
                         "Cumulative_SOC_deviation: {:.4f}".format(SOC_deviation_history), 
                         "Fuel Consumption: {:.4f}".format(env.fuel_consumption), 
                         )
                else: 
                    print(f"Pre-training...Episode: {episode}")
                
                episode_rewards.append(total_reward)
                episode_SOCs.append(env.SOC)
                episode_FCs.append(env.fuel_consumption)
                break 

            state = next_state 
            cnt += 1 
    
    results_dict[reward_factor] = {
        "rewards": episode_rewards, 
        "SOCs": episode_SOCs, 
        "FCs": episode_FCs 
    }
            
    

environment version: 1
maximum steps, simulation is done ... 
Pre-training...Episode: 0
maximum steps, simulation is done ... 
Pre-training...Episode: 1

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer

maximum steps, simulation is done ... 
Episode: 41 Total reward: -4826.884198446341 Explore P: 0.3311 SOC: 1.0000 Cumulative_SOC_deviation: 470.5904 Fuel Consumption: 120.9798
maximum steps, simulation is done ... 
Episode: 42 Total reward: -4762.878586843365 Explore P: 0.3224 SOC: 1.0000 Cumulative_SOC_deviation: 463.6923 Fuel Consumption: 125.9553
maximum steps, simulation is done ... 
Episode: 43 Total reward: -4943.47034397267 Explore P: 0.3140 SOC: 1.0000 Cumulative_SOC_deviation: 481.3771 Fuel Consumption: 129.6991
maximum steps, simulation is done ... 
Episode: 44 Total reward: -4808.97463063938 Explore P: 0.3057 SOC: 1.0000 Cumulative_SOC_deviation: 468.5434 Fuel Consumption: 123.5408
maximum steps, simulation is done ... 
Episode: 45 Total reward: -4917.616170701908 Explore P: 0.2977 SOC: 1.0000 Cumulative_SOC_deviation: 479.0322 Fuel Consumption: 127.2945
maximum steps, simulation is done ... 
Episode: 46 Total reward: -4558.394655812883 Explore P: 0.2899 SOC: 1.0000 Cumulati

maximum steps, simulation is done ... 
Episode: 88 Total reward: -4764.237463024827 Explore P: 0.0983 SOC: 1.0000 Cumulative_SOC_deviation: 464.4177 Fuel Consumption: 120.0609
maximum steps, simulation is done ... 
Episode: 89 Total reward: -4659.335752990775 Explore P: 0.0960 SOC: 1.0000 Cumulative_SOC_deviation: 453.5541 Fuel Consumption: 123.7950
maximum steps, simulation is done ... 
Episode: 90 Total reward: -4785.9859308989335 Explore P: 0.0936 SOC: 1.0000 Cumulative_SOC_deviation: 467.4781 Fuel Consumption: 111.2047
maximum steps, simulation is done ... 
Episode: 91 Total reward: -4685.055530127713 Explore P: 0.0914 SOC: 1.0000 Cumulative_SOC_deviation: 456.6158 Fuel Consumption: 118.8977
maximum steps, simulation is done ... 
Episode: 92 Total reward: -4881.272504198909 Explore P: 0.0892 SOC: 1.0000 Cumulative_SOC_deviation: 476.9452 Fuel Consumption: 111.8205
maximum steps, simulation is done ... 
Episode: 93 Total reward: -4792.807968421967 Explore P: 0.0870 SOC: 1.0000 Cumul

maximum steps, simulation is done ... 
Episode: 135 Total reward: -4823.519236403862 Explore P: 0.0343 SOC: 1.0000 Cumulative_SOC_deviation: 471.9089 Fuel Consumption: 104.4306
maximum steps, simulation is done ... 
Episode: 136 Total reward: -4517.28850461816 Explore P: 0.0336 SOC: 1.0000 Cumulative_SOC_deviation: 441.1851 Fuel Consumption: 105.4374
maximum steps, simulation is done ... 
Episode: 137 Total reward: -4408.102011916895 Explore P: 0.0330 SOC: 1.0000 Cumulative_SOC_deviation: 430.7562 Fuel Consumption: 100.5401
maximum steps, simulation is done ... 
Episode: 138 Total reward: -4672.101852364662 Explore P: 0.0324 SOC: 1.0000 Cumulative_SOC_deviation: 456.4602 Fuel Consumption: 107.4999
maximum steps, simulation is done ... 
Episode: 139 Total reward: -4499.78460740143 Explore P: 0.0318 SOC: 1.0000 Cumulative_SOC_deviation: 439.8795 Fuel Consumption: 100.9897
maximum steps, simulation is done ... 
Episode: 140 Total reward: -4570.656902994395 Explore P: 0.0312 SOC: 1.0000 Cu

maximum steps, simulation is done ... 
Episode: 182 Total reward: -2657.4886026670806 Explore P: 0.0167 SOC: 0.9898 Cumulative_SOC_deviation: 258.5912 Fuel Consumption: 71.5765
maximum steps, simulation is done ... 
Episode: 183 Total reward: -2568.307870145044 Explore P: 0.0165 SOC: 0.9924 Cumulative_SOC_deviation: 249.7680 Fuel Consumption: 70.6283
maximum steps, simulation is done ... 
Episode: 184 Total reward: -2975.474779410259 Explore P: 0.0163 SOC: 1.0000 Cumulative_SOC_deviation: 289.4895 Fuel Consumption: 80.5794
maximum steps, simulation is done ... 
Episode: 185 Total reward: -1578.860443870445 Explore P: 0.0162 SOC: 0.8727 Cumulative_SOC_deviation: 151.6472 Fuel Consumption: 62.3880
maximum steps, simulation is done ... 
Episode: 186 Total reward: -1720.6958043220422 Explore P: 0.0160 SOC: 0.9376 Cumulative_SOC_deviation: 165.4007 Fuel Consumption: 66.6890
maximum steps, simulation is done ... 
Episode: 187 Total reward: -2030.6598908702513 Explore P: 0.0158 SOC: 1.0000 Cu

In [11]:
with open("DDQN3_1.pkl", "wb") as f: 
    pickle.dump(results_dict, f, pickle.HIGHEST_PROTOCOL)

In [12]:
# with open("results/replay_memory_size_effect.pkl", "rb") as f: 
#     data = pickle.load(f)
    
# data